## WMSSRE - custom loss

based on https://www.kaggle.com/sibmike/fast-clear-wrmsse-18ms

In [53]:
import numpy as np 
import pandas as pd

from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix
import gc

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional

import tensorflow as tf
import keras.backend as K



## reload scripts before executing them
%load_ext autoreload
%autoreload 2

from dataset.reduce_memory import reduce_mem_usage


from evaluation.wrmss_loss import WRMSSE_Loss

Using TensorFlow backend.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [465]:
## load data

data_dir = './data/'

# Sales quantities:
sales = pd.read_csv(data_dir+'sales_train_validation.csv')

# Calendar to get week number to join sell prices:
calendar = pd.read_csv(data_dir+'calendar.csv')

# Sell prices to calculate sales in USD:
sell_prices = pd.read_csv(data_dir+'sell_prices.csv')
submission_file = pd.read_csv(data_dir + 'sample_submission.csv')

In [8]:
## this cell runs 12 mins
sales = reduce_mem_usage(sales)
calendar = reduce_mem_usage(calendar)
sell_prices = reduce_mem_usage(sell_prices)

Mem. usage decreased to 95.00 Mb (78.7% reduction)
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Mem. usage decreased to 130.48 Mb (37.5% reduction)


In [10]:
# Dataframe with only last 28 days:
cols = ["d_{}".format(i) for i in range(1914-28, 1914)]
data = sales[["id", 'store_id', 'item_id'] + cols]

# To long form:
data = data.melt(id_vars=["id", 'store_id', 'item_id'], 
                 var_name="d", value_name="sale")

# Add week of year column from 'calendar':
data = pd.merge(data, calendar, how = 'left', 
                left_on = ['d'], right_on = ['d'])

data = data[["id", 'store_id', 'item_id', "sale", "d", "wm_yr_wk"]]

# Add weekly price from 'sell_prices':
data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
data.drop(columns = ['wm_yr_wk'], inplace=True)

# Calculate daily sales in USD:
data['sale_usd'] = data['sale'] * data['sell_price']
data.head()

#this part is correct

,id,store_id,item_id,sale,d,sell_price,sale_usd
0,HOBBIES_1_001_CA_1_validation,CA_1,HOBBIES_1_001,1,d_1886,8.257812,8.257812
1,HOBBIES_1_002_CA_1_validation,CA_1,HOBBIES_1_002,1,d_1886,3.970703,3.970703
2,HOBBIES_1_003_CA_1_validation,CA_1,HOBBIES_1_003,0,d_1886,2.970703,0.000000
3,HOBBIES_1_004_CA_1_validation,CA_1,HOBBIES_1_004,0,d_1886,4.640625,0.000000
4,HOBBIES_1_005_CA_1_validation,CA_1,HOBBIES_1_005,1,d_1886,2.880859,2.880859


In [11]:
# List of categories combinations for aggregations as defined in docs:
dummies_list = [sales.state_id, sales.store_id, 
                sales.cat_id, sales.dept_id, 
                sales.state_id +'_'+ sales.cat_id, sales.state_id +'_'+ sales.dept_id,
                sales.store_id +'_'+ sales.cat_id, sales.store_id +'_'+ sales.dept_id, 
                sales.item_id, sales.state_id +'_'+ sales.item_id, sales.id]


## First element Level_0 aggregation 'all_sales':
dummies_df_list =[pd.DataFrame(np.ones(sales.shape[0]).astype(np.int8), 
                               index=sales.index, columns=['all']).T]

# List of dummy dataframes:
for i, cats in enumerate(dummies_list):
    dummies_df_list +=[pd.get_dummies(cats, drop_first=False, dtype=np.int8).T]
    
# Concat dummy dataframes in one go:
## Level is constructed for free.
roll_mat_df = pd.concat(dummies_df_list, keys=list(range(12)), 
                        names=['level','id'])#.astype(np.int8, copy=False)

# Save values as sparse matrix & save index for future reference:
roll_index = roll_mat_df.index
roll_mat_csr = csr_matrix(roll_mat_df.values)
roll_mat_csr.shape

(42840, 30490)

In [12]:
# Dump roll matrix to pickle:
roll_mat_df.to_pickle('roll_mat_df.pkl')

In [ ]:
# Free some momory:
del dummies_df_list, roll_mat_df
gc.collect()

In [13]:
# Fucntion to calculate S weights:
def get_s(drop_days=0):
    
    """
    drop_days: int, equals 0 by default, so S is calculated on all data.
               If equals 28, last 28 days won't be used in calculating S.
    """
    # Rollup sales:
    d_name = ['d_' + str(i+1) for i in range(1913-drop_days)]
    sales_train_val = roll_mat_csr * sales[d_name].values

    no_sales = np.cumsum(sales_train_val, axis=1) == 0
    sales_train_val = np.where(no_sales, np.nan, sales_train_val)

    # Denominator of RMSSE / RMSSE
    weight1 = np.nanmean(np.diff(sales_train_val,axis=1)**2,axis=1)
    
    return weight1

In [14]:
S = get_s(drop_days=0)
S.shape

(42840,)

In [16]:
# Functinon to calculate weights:
def get_w(sale_usd):
    """
    """
    # Calculate the total sales in USD for each item id:
    total_sales_usd = sale_usd.groupby(
        ['id'], sort=False)['sale_usd'].apply(np.sum).values
    
    # Roll up total sales by ids to higher levels:
    weight2 = roll_mat_csr * total_sales_usd
    
    return 12*weight2/np.sum(weight2)

In [17]:
W = get_w(data[['id','sale_usd']])
W.shape

(42840,)

In [20]:
## de weights worden ook vergeleken met de weights die zijn vrijgegeven om te kijken of we dichtbij zitten


# Predicted weights
W_df = pd.DataFrame(W,index = roll_index,columns=['w'])

# Load the original weights:
W_original_df = pd.read_csv(data_dir+'weights_validation.csv')

# Set new index, calculate difference between original and predicted:
W_original_df = W_original_df.set_index(W_df.index)
W_original_df['Predicted'] = W_df.w
W_original_df['diff'] = W_original_df.Weight - W_original_df.Predicted

# See where we are off by more than e-6
m = W_original_df.Weight.values - W_df.w.values > 0.000001
W_original_df[m]



Level_id  Agg_Level_1  Agg_Level_2    Weight  Predicted  \
level id                                                                       
1     CA               Level2           CA            X  0.442371   0.442370   
3     HOBBIES          Level4      HOBBIES            X  0.128079   0.128075   
      HOUSEHOLD        Level4    HOUSEHOLD            X  0.303335   0.303330   
4     FOODS_1          Level5      FOODS_1            X  0.062625   0.062623   
      FOODS_2          Level5      FOODS_2            X  0.154642   0.154639   
      HOBBIES_1        Level5    HOBBIES_1            X  0.122088   0.122084   
      HOUSEHOLD_1      Level5  HOUSEHOLD_1            X  0.229594   0.229592   
      HOUSEHOLD_2      Level5  HOUSEHOLD_2            X  0.073741   0.073738   
5     CA_HOBBIES       Level6           CA      HOBBIES  0.058855   0.058852   
      CA_HOUSEHOLD     Level6           CA    HOUSEHOLD  0.142772   0.142769   
      TX_HOUSEHOLD     Level6           TX    HOUSEHOLD  0.086420   0.086419   
      WI_HOBBIES       Level6           WI      HOBBIES  0.027931   0.027930   
6     CA_FOODS_2       Level7           CA      FOODS_2  0.057655   0.057654   
      CA_HOBBIES_1     Level7           CA    HOBBIES_1  0.056463   0.056460   
      CA_HOUSEHOLD_1   Level7           CA  HOUSEHOLD_1  0.104863   0.104862   
      CA_HOUSEHOLD_2   Level7           CA  HOUSEHOLD_2  0.037909   0.037907   
      TX_FOODS_1       Level7           TX      FOODS_1  0.016016   0.016015   
      WI_FOODS_2       Level7           WI      FOODS_2  0.062561   0.062560   
      WI_HOBBIES_1     Level7           WI    HOBBIES_1  0.026375   0.026374   
7     CA_2_HOUSEHOLD   Level8         CA_2    HOUSEHOLD  0.037630   0.037629   
      CA_3_HOUSEHOLD   Level8         CA_3    HOUSEHOLD  0.055870   0.055869   
8     WI_2_FOODS_2     Level9         WI_2      FOODS_2  0.030535   0.030534   
9     FOODS_2_029     Level10  FOODS_2_029            X  0.002862   0.002861   
      FOODS_2_183     Level10  FOODS_2_183            X  0.002679   0.002677   
      FOODS_3_586     Level10  FOODS_3_586            X  0.005074   0.005073   

                          diff  
level id                        
1     CA              0.000002  
3     HOBBIES         0.000004  
      HOUSEHOLD       0.000005  
4     FOODS_1         0.000002  
      FOODS_2         0.000004  
      HOBBIES_1       0.000004  
      HOUSEHOLD_1     0.000002  
      HOUSEHOLD_2     0.000003  
5     CA_HOBBIES      0.000003  
      CA_HOUSEHOLD    0.000004  
      TX_HOUSEHOLD    0.000001  
      WI_HOBBIES      0.000001  
6     CA_FOODS_2      0.000001  
      CA_HOBBIES_1    0.000003  
      CA_HOUSEHOLD_1  0.000002  
      CA_HOUSEHOLD_2  0.000002  
      TX_FOODS_1      0.000001  
      WI_FOODS_2      0.000002  
      WI_HOBBIES_1    0.000001  
7     CA_2_HOUSEHOLD  0.000001  
      CA_3_HOUSEHOLD  0.000001  
8     WI_2_FOODS_2    0.000001  
9     FOODS_2_029     0.000001  
      FOODS_2_183     0.000001  
      FOODS_3_586     0.000001

In [21]:
SW = W/np.sqrt(S)


In [22]:
sw_df = pd.DataFrame(np.stack((S, W, SW), axis=-1),index = roll_index,columns=['s','w','sw'])
sw_df.to_pickle('sw_df.pkl')

In [64]:
def rollup(v):
    '''
    v - np.array of size (30490 rows, n day columns)
    v_rolledup - array of size (n, 42840)
    '''
    return roll_mat_csr * v #(v.T*roll_mat_csr.T).T

def rollup_loss(v):
    
    intermediate = roll_mat_csr * K.eval(v)
    
    return tf.convert_to_tensor(intermediate)
# Function to calculate WRMSSE:
def wrmsse(preds, y_true, score_only=False, s = S, w = W, sw=SW):
    '''
    preds - Predictions: pd.DataFrame of size (30490 rows, N day columns)
    y_true - True values: pd.DataFrame of size (30490 rows, N day columns)
    sequence_length - np.array of size (42840,)
    sales_weight - sales weights based on last 28 days: np.array (42840,)
    '''
    
    if score_only:
        return np.sum(
                np.sqrt(
                    np.mean(
                        np.square(rollup(preds.values-y_true.values))
                            ,axis=1)) * sw)/12 #<-used to be mistake here
    else: 
        score_matrix = (np.square(rollup(preds.values-y_true.values)) * np.square(w)[:, None])/ s[:, None]
        score = np.sum(np.sqrt(np.mean(score_matrix,axis=1)))/12 #<-used to be mistake here
        return score, score_matrix
    

In [25]:
# Define fold pass here:
file_pass = './'# '/kaggle/input/fast-wrmsse-and-sw-frame/'

# Load S and W weights for WRMSSE calcualtions:
sw_df = pd.read_pickle(file_pass+'sw_df.pkl')
S = sw_df.s.values
W = sw_df.w.values
SW = sw_df.sw.values

# Load roll up matrix to calcualte aggreagates:
roll_mat_df = pd.read_pickle(file_pass+'roll_mat_df.pkl')
roll_index = roll_mat_df.index
roll_mat_csr = csr_matrix(roll_mat_df.values)
del roll_mat_df

In [26]:
# Predictions:
sub = pd.read_csv(data_dir + 'sample_submission.csv')
sub = sub[sub.id.str.endswith('validation')]
sub.drop(['id'], axis=1, inplace=True)

DAYS_PRED = sub.shape[1]    # 28

# Ground truth:
dayCols = ["d_{}".format(i) for i in range(1914-DAYS_PRED, 1914)]
y_true = sales[dayCols]

In [44]:
# %%timeit -n 100 -r 5
# # n - execute the statement n times 
# # r - repeat each loop r times and return the best

score = wrmsse(sub, y_true, score_only=True)

In [45]:
score

4.5737902294863195

In [466]:
train_sales = reduce_mem_usage(sales)

Mem. usage decreased to 95.00 Mb (78.7% reduction)


In [467]:
# create training data, for now it only contains the sales and no extra features
sales = train_sales.drop(["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"], axis=1).T

# normalize
scaler = MinMaxScaler()
scaler.fit(sales)
sales = scaler.transform(sales)
sales = pd.DataFrame(sales)

In [437]:
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo().astype(np.float32)
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

In [438]:
roll_mat_csr.dtype

dtype('int8')

In [439]:
sparse_tensor = convert_sparse_matrix_to_sparse_tensor(roll_mat_csr)

In [396]:
sparse_tensor.dtype

tf.float32

In [51]:
# create X and y

timesteps = 28
prediction_steps = 1
len_window = timesteps + prediction_steps

nr_training_days = sales.shape[0]
nr_sets = nr_training_days - len_window + 1

base, predictions = [], []

for i in range(nr_sets):
    samples = sales.iloc[i:i+timesteps]
    pred = sales.iloc[i+timesteps]
    base.append(samples.to_numpy())
    predictions.append(pred.to_numpy())
    
X = np.array(base)
y = np.array(predictions)

del base, predictions

In [54]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor = 0.1, patience = 2 , verbose = 1, mode= 'min', min_lr = 0.000001)


In [534]:

def rollup_loss(v):

#     intermediate = sparse_tensor * v
    intermediate = tf.sparse.sparse_dense_matmul(sparse_tensor,tf.transpose(v))
    return intermediate

# Function to calculate WRMSSE during training:
# make sure that s = S, w = W, sw=SW) are available
def wrmsse_loss(y_true, y_pred):
    '''
    preds - Predictions: pd.DataFrame of size (30490 rows, N day columns)
    y_true - True values: pd.DataFrame of size (30490 rows, N day columns)
    sequence_length - np.array of size (42840,)
    sales_weight - sales weights based on last 28 days: np.array (42840,)
    '''
        
#     return K.sum(root_mean_squared_scaled_error(y_true,y_pred) * SW)  / 12 

    return K.sum(
            K.sqrt(
                K.mean(
                    K.square(rollup_loss(y_pred-y_true))
                        ,axis=1)) * SW)/12 #<-used to be mistake here


In [535]:
def root_mean_squared_scaled_error(y_true,y_pred):
    
    sample_length = 28 ## 28 historical days ..
    forecasting_horizon = 1 ## we are predicting 1 day...
    upper_bound = sample_length + forecasting_horizon
    lower_bound = sample_length + 1 - 1
    numerator =  K.sum(K.square(y_true[lower_bound:upper_bound]-y_pred[lower_bound:upper_bound]))
    lower_bound = 2 - 1
    ## normally we would only count the denominator starting when the product is actively sold
    ## I don't see how we can achieve that so I did not do it.
    denominator = (1/(sample_length - 1 )) * K.sum(K.square(y_true[lower_bound:sample_length] - y_true[lower_bound-1:sample_length-1]))
    value_to_be_sqrt = (1/forecasting_horizon) * (numerator/denominator)
    result = K.sqrt(value_to_be_sqrt)
    return result

In [536]:

n_features = X.shape[2]

model = Sequential()
model.add(Bidirectional(LSTM(20, return_sequences=True, input_shape=(timesteps, n_features))))
model.add(Bidirectional(LSTM(10)))
model.add(Dense(30490))
model.compile(optimizer='adam', loss=wrmsse_loss, metrics=['mse'])

# callbacks=[WandbCallback(data_type="image", validation_data=(X_test, y_test), labels=character_names)

In [ ]:
model.fit(X, y, batch_size=32, epochs=10, callbacks=[reduce_lr], verbose=1)


In [ ]:
# get predictions

for i in range(28):    
    # get input for prediction by selecting last 28 days from sales
    X_pred = []
    X_pred.append(sales.iloc[-timesteps:].to_numpy())
    X_pred = np.array(X_pred)
    
    # get prediction
    prediction = model.predict(X_pred)
    # add prediction to sales so that it can be used for next prediction
    sales.loc[sales.shape[0]] = prediction[0]
    
predictions = sales.iloc[-28:]
predictions = scaler.inverse_transform(predictions)
predictions = np.round(np.abs(predictions))
predictions = pd.DataFrame(predictions).T

In [469]:
# create submission file
predictions_copy = predictions
final_submission = pd.concat([predictions, predictions_copy])
final_submission.reset_index(drop=True, inplace=True)
final_submission = final_submission.astype(int)
final_submission.insert(0, 'id', submission_file['id'])
final_submission.columns = ['id'] + [f"F{i}" for i in range(1, 29)]

final_submission.to_csv('submission_wrmsse.csv', index=False)